In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
pip install PyPDF2

     |████████████████████████████████| 77 kB 4.1 MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61100 sha256=3607f709e82a5ab734c1e0602d0b5a45f34631149cfbe422deec1b6c4e103d61
  Stored in directory: /root/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built PyPDF2


In [ ]:
import PyPDF2
import re
from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.parsing.preprocessing import STOPWORDS, RE_WHITESPACE
from gensim.corpora.dictionary import Dictionary
from gensim.models import KeyedVectors

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Removes stopwords from a list of token
def remove_stopwords(tokens, stopwords=STOPWORDS):
    return [token for token in tokens if token not in stopwords]

In [ ]:
# Removes tokens that appear less than minsize times
def remove_short(tokens, minsize=2):
    return [token for token in tokens if len(token) >= minsize]

In [ ]:
# Removes > 1 whitespaces b/w words
def strip_multiple_whitespaces(s):
    return RE_WHITESPACE.sub(" ", s)

In [ ]:
model = KeyedVectors.load_word2vec_format('drive/My Drive/Colab Notebooks/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
filename = "dataset.pdf"

In [ ]:
file = PyPDF2.PdfFileReader('drive/My Drive/Colab Notebooks/' + filename)
Numpages = file.getNumPages()
text = ""
for j in range(0,Numpages):
    PageObj = file.getPage(j)
    text = text + " " + PageObj.extractText()

In [ ]:
#paragraphs = re.split ('\.|\?|\!', text) # Sentence break
paragraphs = re.split ('\.\n', text) # Paragraph break
paragraphs = [re.sub('[^-\.!\?,;a-zA-Z0-9]+', ' ', paragraph.strip()) for paragraph in paragraphs]
original_para = paragraphs.copy()

In [ ]:
lm = WordNetLemmatizer()
paragraphs = [re.sub('[^-a-zA-Z0-9]+', ' ', paragraph.lower()) for paragraph in paragraphs]
for paragraph in paragraphs:
    #paragraph = [lm.lemmatize(word) for word in paragraph if not word in stopwords.words('english')]
    paragraph = [word for word in paragraph if not word in stopwords.words('english')]

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)


In [ ]:
query = "Nehru"
query = query.split()

In [ ]:
query = [q for q in query if not q in stopwords.words('english')]

In [ ]:
# Cases
# 1 -> [query]
# 2 -> [word1] + [word2] + ... + [word_n]
# 3 -> [word1_word2_..._worn_n]
query_vector = [(q,1) for q in query]
for word in query:
  word = str(word)
  try:
    similar_vector = model.most_similar ([word], topn = 20)
  except KeyError:
    similar_vector = []
    print("",end="")
  query_vector.extend (similar_vector)

In [ ]:
query_vector = [(str(query_vector[i][0]).replace ("_"," "), query_vector[i][1]) for i in range (len(query_vector))]

In [ ]:
query_vector.sort (key = lambda x: -x[1])
paragraph_indices = [(i,0) for i in range (len(paragraphs))]
for query in query_vector:
  for i in range (len(paragraphs)):
    paragraph_indices [i] = (i, paragraph_indices[i][1] + paragraphs[i].lower().count(query[0].lower()))
paragraph_indices.sort (key = lambda x: -x[1])

In [ ]:
print (paragraph_indices[:10])
print (query_vector)

[(765, 30), (768, 16), (694, 11), (619, 9), (634, 9), (626, 8), (645, 8), (727, 8), (586, 7), (593, 7)]
[('Nehru', 1), ('Jawaharlal Nehru', 0.7297710180282593), ('Pandit Jawaharlal Nehru', 0.686538815498352), ('Pandit Nehru', 0.6833187341690063), ('Lal Bahadur Shastri', 0.6741498708724976), ('Indira Gandhi', 0.6672089099884033), ('Ambedkar', 0.6314840912818909), ('Mahatama Gandhi', 0.6306662559509277), ('Narasimha Rao', 0.6273825168609619), ('Jawahar Lal Nehru', 0.6264326572418213), ('Jawaharlal', 0.6253728866577148), ('Chacha Nehru', 0.6252062320709229), ('prime minister Jawaharlal Nehru', 0.6234115958213806), ('Rajiv Gandhi', 0.6233093738555908), ('Tagore', 0.6217092275619507), ('Maulana Azad', 0.6209678649902344), ('Jinnah', 0.6170781254768372), ('Vajpayee', 0.6167270541191101), ('Motilal Nehru', 0.6164277791976929), ('Sardar Patel', 0.6139699220657349), ('Gandhiji', 0.6129738092422485)]


In [ ]:
max_relevant_para = 5

In [ ]:
for i in range (max_relevant_para):
  if paragraph_indices [i][1] > 0:
    print (original_para [paragraph_indices [i][0]])
    for i in range (10):
      print ()

114 46 BIBLIOGRAPHY AND NOTES352Kerrin Gr fin Schwerin, Indirekte Herrschaft und Reformpolitik im indischenF rstenstaat Hyderabad, 1853 1911 Wiesbaden, 1980 Moin Shakir, Khilafat to Partition A Survey of Major Political Trends among Indian Muslims during 1919 1947 New Delhi, 1970 B.Pattabhi Sitaramayya, History of the Indian National Congress, 2 vols Bombay, 1935 47 David Washbrook, The Emergence of Provincial Politics The Madras Presidency, 1870 1920 Cambridge, 1976 The partition of IndiaAbul Kalam Azad, India Wins Freedom Bombay, 1959 K.K.Aziz, Rahmat Ali A Biography Lahore, 1986 Alan Campbell-Johnson, Mission with Mountbatten London, 1953 H.V.Hodson, The Great Divide New York, 1971 Nicholas Mansergh ed. , The Transfer of Power, vols 3 8 London, 1971 9 V.P.Menon, The Transfer of Power in India Bombay, 1957 Penderel Moon ed. , Wavell The Viceroy s Journal London, 1973 Robin J.Moore, Churchill, Cripps and India, 1939 1945 Oxford, 1979 Robin J.Moore, Escape from Empire The Attlee Govern

In [ ]:
model.most_similar (['Bollywood'],topn=30)

In [ ]:
model.most_similar (['Mamata']+['Banerjee'])

[('Mamata_Banerjee', 0.8615977764129639),
 ('Bhattacharjee', 0.8172184228897095),
 ('Mayawati', 0.7947120666503906),
 ('Ms_Banerjee', 0.7882030606269836),
 ('Trinamool', 0.7864018678665161),
 ('Chatterjee', 0.7811701893806458),
 ('Mamta_Banerjee', 0.7778410911560059),
 ('Trinamul', 0.7742266654968262),
 ('Deshmukh', 0.7729364633560181),
 ('Chakraborty', 0.7718198299407959)]

In [ ]:
model.most_similar (['Mamata_Banerjee'])

[('Mamata', 0.8583166599273682),
 ('Mayawati', 0.8086827993392944),
 ('Mamta_Banerjee', 0.7967199087142944),
 ('Ms_Banerjee', 0.795053243637085),
 ('chief_Mamata_Banerjee', 0.7819643020629883),
 ('Trinamool_Congress', 0.7801012992858887),
 ('Jayalalithaa', 0.7720375061035156),
 ('Buddhadeb_Bhattacharjee', 0.7658127546310425),
 ('Banerjee', 0.7628343105316162),
 ('Trinamool', 0.7548649311065674)]

In [ ]:
model.most_similar (['Chief'] + ['Minister'])

[('Minsiter', 0.6639981269836426),
 ('MInister', 0.6590216159820557),
 ('Minis_ter', 0.6244001984596252),
 ('Mini_ster', 0.6225288510322571),
 ('Mnister', 0.6224889755249023),
 ('Deputy', 0.6201450228691101),
 ('Minister_Datuk_Lajim', 0.6192092299461365),
 ('Miniser', 0.6160105466842651),
 ('Mininster', 0.6013134717941284),
 ('Secretary', 0.6004642248153687)]

In [ ]:
model.most_similar (['chief_minister'])

KeyError: ignored